In [11]:
import os
import sys, traceback
import argparse
import csv
from datetime import datetime
from functools import reduce
import glob
from itertools import groupby
import numpy as np
import pandas as pd
from functools import partial
from multiprocessing import Pool
from pybedtools import BedTool

In [ ]:
# TO RUN

# python script input_file sample_id

# python /dors/capra_lab/fongsl/enh_age/bin/age_enhancers.py UBERON_0002372_tonsil_expressed_enhancers.bed UBERON0002372


###
#   arguments
###
arg_parser = argparse.ArgumentParser(description="Calculate enhancer age.")

arg_parser.add_argument("region_file_1", help='bed file 1 (enhancers to age) w/ full path')

arg_parser.add_argument("sample_id", help='str label for files')

arg_parser.add_argument("-i", "--iters", type=int, default=100,
                        help='number of simulation iterations; default=100')

arg_parser.add_argument("-s", "--species", type=str, default='hg19', choices=['hg19', 'hg38', 'mm10'],
                        help='species and assembly; default=hg19')

arg_parser.add_argument("-n", "--num_threads", type=int,
                        help='number of threads; default=SLURM_CPUS_PER_TASK or 1')

arg_parser.add_argument("--print_counts_to", type=str, default=None,
                        help="print expected counts to file")

args = arg_parser.parse_args()

TEST_ENH = args.region_file_1
SAMPLE_ID = args.sample_id
COUNT_FILENAME = args.print_counts_to
ITERATIONS = args.iters
SPECIES = args.species
TEST_PATH = "/".join(TEST_ENH.split("/")[:-1])

AGE_OUTFILE ="%s/%s_enh_ages.bed" %(TEST_PATH, SAMPLE_ID)
SHUFFLE_ID = "Shuf-%s" % SAMPLE_ID

# calculate the number of threads

if args.num_threads:
    num_threads = args.num_threads
else:
    num_threads = int(os.getenv('SLURM_CPUS_PER_TASK', 1))

In [12]:
###
#   functions
###

def loadConstants(species):  # note chrom.sizes not used in current implementation | 2018.10.29
    return {'hg19': ("/dors/capra_lab/users/bentonml/data/dna/hg19/hg19_blacklist_gap.bed", "/dors/capra_lab/data/dna/human/hg19/hg19_trim.chrom.sizes"),
            'hg38': ("/dors/capra_lab/users/bentonml/data/dna/hg38/hg38_blacklist_gap.bed", "/dors/capra_lab/data/dna/human/hg38/hg38_trim.chrom.sizes"),
            'mm10': ("/dors/capra_lab/users/bentonml/data/dna/mm10/mm10_blacklist_gap.bed", "/dors/capra_lab/data/dna/mouse/mm10/mm10_trim.chrom.sizes")
            }[species]


def enh_age(test_enh, sample_id, test_path, species):
    os.chdir(test_path)
    
    cut_file = "standard_%s.bed" % sample_id # standard.bed
    cut_cmd = "cut -f 1-3 %s > ./%s" % (test_enh, cut_file) # standardize the input file
    
    os.system(cut_cmd)
    
    chr_cmd = "awk '{print >$1\"_%s_temp.bed\"}' %s" % (sample_id, cut_file) # split test into chrN.bed files
    os.system(chr_cmd)
    
    rm_cut_file = "rm ./%s" % cut_file
    os.system(rm_cut_file)
    
    enh_chr_list = glob.glob("%s/chr*_%s_temp.bed" % (test_path, sample_id)) # glob chromosomes

    for enh_chr in enh_chr_list: # intersect test chrN.bed w/ syntenic block
        chr_num = (enh_chr.split("/")[-1]).split("_")[0]
        
        syn_path = "/dors/capra_lab/data/ucsc/%s/synteny_age_bkgd_%s/" %(species, species)
        syn_file= ("%s%s_syn_age.bed" % (syn_path, chr_num))
        
        outfile = "%s/%s_%s_ages.bed" %(test_path, chr_num, sample_id,)
        bed_cmd = "bedtools intersect -a %s -b %s -wao > %s" % (enh_chr, syn_file, outfile)
        
        os.system(bed_cmd)

    concat_cmd = "cat %s/chr*_%s_ages.bed > %s/%s_enh_ages.bed" % (test_path, sample_id, test_path, sample_id)#concatenate chromosomes
    print("concatenating aged enhancer chromosome files", concat_cmd)
    os.system(concat_cmd)
    
    clean_up_temp = "rm %s/chr*_%s_*.bed" % (test_path, sample_id) # remove old files
    os.system(clean_up_temp)

def break_tfbs(sample_id, test_path):
    print(sample_id, test_path)
    outpath = "%s/breaks/" % test_path # mkdir ./break/
    cmd = "mkdir %s" % outpath
    os.system(cmd)

    df = pd.read_csv("%s/%s_enh_ages.bed"%(test_path, sample_id),\
                         sep = '\t', header = -1 , low_memory=False) # open up the bed file and assemble breaks.

    df.columns = ["chr_enh", # rename the columns
                  "start_enh", 
                  "end_enh",
                  "chr_syn",
                  "start_syn",
                  "end_syn",
                  "strand",
                  "ref",
                  "num_species",
                  "len_syn",
                  "mrca",
                  "patr",
                  "len_syn_overlap"]
    
    df["test_id"] = df["chr_enh"] + ":" + df["start_enh"].map(str) + "-" + df["end_enh"].map(str) # add a test_id
    
    no_syn = df.loc[df["len_syn_overlap"]<1] # remove "." coordinates non-overlapping with syntenic blocks

    if len(no_syn) >1: # save all coordinates not overlapping syntenic blocks
        no_syn.to_csv("%s%s_no_syn_alignment_tiles.txt" % (outpath, sample_id),\
                      sep = '\t', header = True, index = False) # write non-overlapping regions
    df = df.loc[df["len_syn_overlap"]>0] # remove all the records that do not overlap syntenic blocks

    print("unique enhancers =", len(df["test_id"].unique())) # count the number of enhancers
    
    print("# rows to reduce and assemble breaks = ", len(df.drop_duplicates())) # drop duplicates
    df.drop_duplicates()

    #####
    # prepare to join breaks
    #####

    df["mrca"] = df["mrca"].astype(float).round(3) # round MRCA distance

    id_list = df["test_id"].unique() # get enh_ids

    #####
    # start the break assembly
    #####
    start = datetime.now()
    print("Start assembling breaks", start)

    df_dict = {} # collect all the information in one dictionary

    val = 0
    write_val = 500

    for i in id_list:

        test = df[["chr_enh",  # subset dataset to one enhancer ID
                   "start_enh", 
                   "end_enh",
                   "chr_syn",
                   "start_syn",
                   "end_syn",
                   "mrca",
                   "len_syn_overlap",
                   "test_id"]].loc[df["test_id"]==i].drop_duplicates().sort_values(["chr_syn",
                   "start_syn",
                   "end_syn"]).reset_index()

        new_data = test[["chr_enh",  # dataframe for reassembled breaks
                         "start_enh", 
                         "end_enh",
                         "test_id",
                         "chr_syn"]].drop_duplicates()

        collect_df = pd.DataFrame()

        age_seg = [(age, sum(1 for i in rows)) for age,rows in groupby(test["mrca"])]

        start = test["start_enh"].astype(int)
        end = test["end_enh"].astype(int)

        df_index = 0
        seg_index = 0
        core_age = max(i for i, k in age_seg) # get the max age
        core_age = round(core_age, 3) # REDUNDANCY, but just in case

        for tup in age_seg:
            age, idx = tup
            age = round(age, 3) # REDUNDANCY, but just in case


            if len(age_seg)== 1: # simple enhancers

                new_data["start_syn"] = start
                new_data["end_syn"] = end
                new_data["mrca_seg"] = age
                new_data["seg_index"] = 0 # index syntenic segments
                new_data["core"] = 1 # binary core measure
                new_data["core_remodeling"] = 0 # binary core remodeling measure
                collect_df= collect_df.append(new_data)

            else: # complex enhancers

                new_data["seg_index"]= seg_index # index syntenic segments
                new_data["core_remodeling"] = 1 # binary core remodeling measure

                if age == core_age: # mark the core
                    new_data["core"] = 1
                    
                else:
                    new_data["core"] = 0

                if seg_index == 0: # trim first syntenic block to start
                    new_data["start_syn"] = start
                    new_data["end_syn"] = test.loc[idx-1, "end_syn"]

                    new_data["mrca_seg"] = age
                    collect_df= collect_df.append(new_data)

                elif seg_index == len(age_seg)-1: # trim last syntenic block
                    new_data["mrca_seg"] = age
                    new_data["start_syn"] = test.loc[df_index, "start_syn"]
                    new_data["end_syn"] = end
                    collect_df= collect_df.append(new_data)

                else: # deal with all the blocks in between first and last syntenic block
                    new_data["mrca_seg"] = age
                    new_data["start_syn"]= test.loc[df_index, "start_syn"]
                    new_data["end_syn"]= test.loc[df_index + idx -1, "end_syn"]
                    collect_df= collect_df.append(new_data)

                df_index +=idx # go to next index
                seg_index +=1 # count age segments

        df_dict[i] = collect_df
        val +=1

        if val == write_val: #periodically write the file
            temp = pd.concat(df_dict.values(), sort = True)
            temp = temp[['chr_syn',
                         'start_syn',
                         'end_syn',
                         'test_id',
                         'chr_enh',
                         'start_enh',
                         'end_enh',
                         'seg_index',
                         'core_remodeling',
                         'core',
                         'mrca_seg']]
            temp_out = "%s%s_age_breaks.bed" % (outpath, sample_id)
            with open(temp_out, 'a') as f:
                temp.to_csv(f, sep = '\t', header = False, index = False) # write last enhancers   

            write_val += 500 # re-up write val
            df_dict = {} # collect all the information in one dictionary
            
    temp = pd.concat(df_dict.values(), sort = True)
    temp = temp[['chr_syn',
                 'start_syn',
                 'end_syn',
                 'test_id',
                 'chr_enh',
                 'start_enh',
                 'end_enh',
                 'seg_index',
                 'core_remodeling',
                 'core',
                 'mrca_seg']]
    temp_out = "%s%s_age_breaks.bed" % (outpath, sample_id)
    with open(temp_out, 'a') as f:
        temp.to_csv(f, sep = '\t', header = False, index = False) # write last enhancers   

    print("Finished assembling breaks", datetime.now())
    
def tfbs_density(sample_id, test_path):

    in_path = "%s/breaks/" % test_path 

    out_path = "%s/tfbs/"% test_path

    #####
    # prepare workspace
    #####
    core_breaks_file = "%s%s_age_breaks.bed"% (in_path, sample_id)

    temp_val = 0

    tfbs_file = "/dors/capra_lab/data/encode/midpeaks_wgEncodeRegTfbsClusteredV3.bed" # TFBS data, 30bp trimmed ChIP-Peaks

    mkdir_cmd = "mkdir %s" %out_path # make the out_path directory
    os.system(mkdir_cmd)

    ########
    # PART 1 - sort bed file
    ########

    sort_cmd = "sort -k5,5 -k2,2 -k3,3 -k6,6 -k7,7 %s >t && mv t %s" % (core_breaks_file, core_breaks_file)
    os.system(sort_cmd)

    ########
    # PART 2 - Bed intersect file with TFBS
    ########

    outfile_wao = "%s%s_x_raw_tfbs_midpeak.bed"% (out_path, sample_id)

    bed_cmd = "bedtools intersect -a %s -b %s -wao > %s" % (core_breaks_file, tfbs_file, outfile_wao)

    os.system(bed_cmd)

    ########
    # open the sample_id file
    ########

    enh = pd.read_csv(outfile_wao, header= -1, sep= '\t',low_memory = False)
    print(len(enh), "raw df len")
    ########
    # reformat file to get columns/data you want
    ########

    print("the number syntenic blocks that did not overlap TFBS" ,\
          len(enh[20].loc[enh[20]==0]))

    #select the columns you want for analyzing break density
    enh = enh[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 20]].drop_duplicates()

    # rename the columns
    enh.columns = ["chr_syn", 
                   "start_syn", 
                   "end_syn", 
                   "enh_id", 
                   "chr_enh", 
                   "start_enh", 
                   "end_enh", 
                   "seg_index", 
                   "core_remodeling", 
                   "core", 
                   "mrca",
                   "chr_tf", 
                   "start_tf", 
                   "end_tf", 
                   "tf",  
                   "len_tfbs_overlap"]

    # assign a unique syn id
    enh["syn_id"] = enh["chr_syn"] + ":" + enh["start_syn"].map(str) + "-" + enh["end_syn"].map(str)

    # assign unique tf_id
    enh["tf_id"] = enh["chr_tf"] + ":" + enh["start_tf"].map(str) + "-" + enh["end_tf"].map(str) +"_"+ enh["tf"]

    print("the number of unique enhancers", len(enh["enh_id"].unique()))

    enh_tf_list = enh["enh_id"].unique() # make a list of unique enhancer ids

    if temp_val > 0:
        print("remove enh_ids that have already been done", temp_val)
        enh_tf_list = enh_tf_list[(temp_val):]
        val = temp_val
    else:
        enh_tf_list = enh_tf_list
        val = 0

    # transform the dtypes for quantitative values
    enh[["start_enh", 
         "end_enh",
         "start_tf",
         "start_syn", 
         "end_syn", 
         "end_tf", 
         "seg_index", 
         "core_remodeling", 
         "core","len_tfbs_overlap"]] = enh[["start_enh", 
         "end_enh",
         "start_tf",
         "start_syn", 
         "end_syn", 
         "end_tf", 
         "seg_index", 
         "core_remodeling", 
         "core","len_tfbs_overlap"]].astype(int)

    enh["mrca"] = enh["mrca"].astype(float).round(3)
    enh["enh_len"]= enh["end_enh"]- enh["start_enh"] # enhancer lengths
    enh["syn_len"]= enh["end_syn"]- enh["start_syn"] # syntenic lengths

    ########
    # calculate TFBS density
    ########

    ########
    # count tfs associated with enhancers
    ########
    tfenh_base = enh[["chr_enh", "start_enh", "end_enh", "enh_id", "enh_len"]].drop_duplicates()

    tfenh = enh.loc[enh["len_tfbs_overlap"]>=6].groupby(["enh_id", "enh_len"])["tf"].count().reset_index()

    tfenh.columns=['enh_id','enh_len','enh_tf_count'] # rename columns

    tfenh["enh_tf_den"] = tfenh["enh_tf_count"].divide(tfenh["enh_len"]) # enhancer tfbs density
    tfenh = tfenh[['enh_id','enh_tf_count', 'enh_tf_den']] # keep only essential info from df

    # count uniq tfs associated with enhancers

    tfenhunq = enh.loc[enh["len_tfbs_overlap"]>=6].groupby(["enh_id", "enh_len"])["tf"].unique().reset_index()

    tfenhunq["enh_tf_unq_count"] = tfenhunq["tf"].apply(lambda x: len(x))# count the number of unique TFs

    tfenhunq["enh_tf_unq_den"] = tfenhunq["enh_tf_unq_count"].divide(tfenhunq["enh_len"])# enhancer tfbs density
    tfenhunq = tfenhunq[['enh_id','enh_tf_unq_count', 'enh_tf_unq_den']] # keep only essential info from df

    # reduce base, tfden, tfuniqden dataframes

    enhdfs = [tfenh_base, tfenh, tfenhunq]
    enh_merged = reduce(lambda left,right: pd.merge(left,right,on=["enh_id"],
                                                how='left'), enhdfs)

    print("# total enhacers", len(tfenh_base))
    print("# enh overlapping TF >=6bp", len(tfenh)) # exclude any enhancers w/o tfbs overlapping >=6bp
    print("# unq enh overlapping TF >=6bp", len(tfenhunq)) # exclude any enhancer w/o uniq tfbs overlapping >=6bp
    print("len merged enh", len(enh_merged)) # the left-merged df

    enh_merged.to_csv("%s%s_enh_tfbs_density.bed" % (out_path, sample_id), sep= '\t', header = True, index = False)

    ########
    # count syn tfs associated with syntenic blocks
    ########
    tfsyn_base = enh[["enh_id","chr_syn", "start_syn", "end_syn", "syn_id",
              "seg_index", "core_remodeling", "core", "mrca", "syn_len"]].drop_duplicates()

    tfsyn = enh.loc[enh["len_tfbs_overlap"]>=6].groupby(["enh_id",\
                                                         "syn_id","seg_index", "syn_len"])["tf"].count().reset_index()
    

    tfsyn.columns=["enh_id", "syn_id","seg_index", "syn_len", 'syn_tf_count'] # rename columns

    tfsyn["syn_tf_den"] = tfsyn["syn_tf_count"].divide(tfsyn["syn_len"])

    # count uniq syn tfs associated with syntenic blocks

    tfsynunq = enh.loc[enh["len_tfbs_overlap"]>=6].groupby(["enh_id","syn_id","seg_index","syn_len"])["tf"].unique().reset_index()


    tfsynunq["syn_tf_unq_count"] = tfsynunq["tf"].apply(lambda x: len(x))# count the number of unique TFs
    tfsynunq["syn_tf_unq_den"] = tfsynunq["syn_tf_unq_count"].divide(tfsynunq["syn_len"])

    syndfs = [tfsyn_base, tfsyn, tfsynunq]

    syn_merged = reduce(lambda left,right: pd.merge(left,right,on=['enh_id','syn_id', 'seg_index', 'syn_len'],
                                                how='left'), syndfs)

    print("# total syn blocks", len(tfsyn_base))
    print("# total syn blocks overlapping tf >= 6bp", len(tfsyn)) # exclude any syn w/o tfbs overlapping >=6bp
    print("# total unq syn blocks overlapping tf >= 6bp", len(tfsynunq)) # exclude any syn w/o uniq tfbs overlapping >=6bp
    print("len merged syn", len(syn_merged)) # the left-merged df

    syn_merged.to_csv("%s%s_syn_tfbs_density.bed" % (out_path, sample_id), sep= '\t', header = True, index = False)

    print("Finished calculating TFBS density", datetime.now())  

    
def calculateExpected(test_enh, shuffle_id, test_path, species, iters):
    BLACKLIST, CHROM_SZ = loadConstants(species)  # note CHROM_SZ not used
    exp_sum = 0

    shuffle_path = "%s/shuffle" % test_path # make a shuffle path file
    os.system("mkdir %s" % shuffle_path)

    rand_file = BedTool(test_enh).shuffle(genome='hg19', excl=BLACKLIST, chrom=True, noOverlapping=True) # shuffle bed
    rand_out = '%s/rand_file_%s.bed'% (shuffle_path, shuffle_id) # make shuffle file
    rand_file.saveas(rand_out)# save shuffle

    enh_age(rand_out, shuffle_id, shuffle_path, species) # age the shuffles

    break_tfbs(shuffle_id, shuffle_path) # measure shuffle breaks
    
    #tfbs_density(shuffle_id, shuffle_path) # intersect shuffles with ENCODE TFBS
###
#   main
###

In [13]:
TEST_ENH = '/dors/capra_lab/projects/enhancer_ages/klein2018/data/merged.bed'
SHUFFLE_ID = 'Shuf_test2-klein'
TEST_PATH = '/dors/capra_lab/projects/enhancer_ages/klein2018/data'
SPECIES = 'hg19'
ITERATIONS = 3
num_threads= 1

In [14]:

def main(argv):
    print('python {:s} {:s}'.format(' '.join(sys.argv), str(datetime.now())[:20]))
    
#    enh_age(TEST_ENH, SAMPLE_ID, TEST_PATH, SPECIES) # age enhancers
 #   break_tfbs(SAMPLE_ID, TEST_PATH)
  #  tfbs_density(SAMPLE_ID, TEST_PATH)
    
    # create pool and run simulations in parallel
    pool = Pool(num_threads)
    partial_calcExp = partial(calculateExpected, BedTool(TEST_ENH), SHUFFLE_ID, TEST_PATH, SPECIES)
    exp_sum_list = pool.map(partial_calcExp, [i for i in range(ITERATIONS)])
    
if __name__ == "__main__":
    main(sys.argv[1:])
    

python /accre/arch/easybuild/software/BinDist/Anaconda3/5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py -f /gpfs22/home/fongsl/.local/share/jupyter/runtime/kernel-faad2413-db6e-49b2-88ba-96adf17298a8.json 2019-08-02 14:40:20.
concatenating aged enhancer chromosome files cat /dors/capra_lab/projects/enhancer_ages/klein2018/data/shuffle/chr*_Shuf_test2-klein_ages.bed > /dors/capra_lab/projects/enhancer_ages/klein2018/data/shuffle/Shuf_test2-klein_enh_ages.bed
concatenating aged enhancer chromosome files cat /dors/capra_lab/projects/enhancer_ages/klein2018/data/shuffle/chr*_Shuf_test2-klein_ages.bed > /dors/capra_lab/projects/enhancer_ages/klein2018/data/shuffle/Shuf_test2-klein_enh_ages.bed
Shuf_test2-klein /dors/capra_lab/projects/enhancer_ages/klein2018/data/shuffle
Shuf_test2-klein /dors/capra_lab/projects/enhancer_ages/klein2018/data/shuffle
unique enhancers = 797
unique enhancers = 797
# rows to reduce and assemble breaks =  12628
# rows to reduce and assemble breaks =  1262